# NIRCam Demo with Eureka!
## Authors: Megan Mansfield and Laura Mayorga
### Hello! This notebook will walk you through a full demonstration of using the [Eureka!](https://github.com/kevin218/Eureka) pipeline to analyze a NIRCam data set.
### Before running this Jupyter Notebook, please follow the steps in "README.md" to set up an environment for this demonstration and install Eureka. Note that this demonstration is using a fixed version of Eureka! to ensure that no future updates break the example here, so even if you have the current version of Eureka! installed on your own machine you'll need to follow the set-up steps to ensure this demo works properly.
#### One more note: this code can be run on either the full data set, contained in "NIRCam_full_data", or the tiny data set, contained in "NIRCam_tiny_data". (For more information on these data sets, see [the NIRCam simulated data](https://stsci.app.box.com/s/ctuol6orkulkrytbt7ajbd5653j93tg4).) The benefit of using the tiny data set is it will run much more quickly, but it starts with Stage 2 outputs. Additionally, the tiny data set does not contain a transit, so if you are using this data set the only relevant Eureka! steps to run will be Stage 3 and Stage 4. The full data set will take longer to run but starts with Stage 1 outputs and contains a transit, so it can be run all the way through Stages 2-6.
### After downloading this code, you will also need to download the [NIRCam simulated data](https://stsci.app.box.com/s/ctuol6orkulkrytbt7ajbd5653j93tg4). To run the full demo, download the files in astrophysical-only_model/Stage 1 Output/Long Wavelength Spectroscopy. To run the tiny demo, download the file in tiny/Stage2. For both sets, you will also need to download the ancillary files located in tiny/ancil. Place these ancillary files in a directory called "ancil".

## To start: let's import packages!

In [ ]:
import os
os.environ["CRDS_PATH"]="crds_cache/"
os.environ["CRDS_SERVER_URL"]="https://jwst-crds.stsci.edu"

import sys
sys.path.append('./')
sys.path.insert(0,'./')
import eureka.S1_detector_processing.s1_process as s1
import eureka.S2_calibrations.s2_calibrate as s2
import eureka.S3_data_reduction.s3_reduce as s3
import eureka.S4_generate_lightcurves.s4_genLC as s4
import eureka.S5_lightcurve_fitting.s5_fit as s5
import eureka.S6_planet_spectra.s6_spectra as s6

eventlabel = 'nircam_wfss_template'
ecf_path = './ecf/'

## Eureka! is divided into six "Stages". Each Stage has a corresponding Eureka Control File, or .ecf file. These files are named "S2_eventlabel.ecf" where the first half of the name (S2, S3, etc.) refers to the Stage that .ecf file interacts with, and the second half refers to the "eventlabel" keyword we defined above. We'll discuss each Stage in Eureka! in turn here.

## Stage 1: Correcting detector-level effects and fitting the up-the-ramp slope.

### This stage isn't included in the demo because it is currently identical to the standard JWST pipeline. However, the full Eureka! package includes the capability to add your own Stage 1 routines. One example of a case where you might want to do this is correcting for 1/f noise, which is not currently addressed by the JWST pipeline.

## Stage 2: Additional pre-spectral-extraction steps, such as flat fielding and unit conversions.

### Note: this step only needs to be run for the full data set. The tiny data set included here starts with Stage 2 outputs, so if you're using that data set skip this step.

### The .ecf file for this stage primarily contains keywords that you don't need to worry about changing, in many cases because they are steps that don't need to be performed for NIRCam. Here I'll walk through the .ecf keywords that you'll need to pay attention to:

#### 1. suffix - This specifies the suffix of the files you're feeding in to Stage 2. For this simulated data set, the suffix is "rateints".
#### 2. skip_flat_field - Setting this to "true" will skip the flat fielding step, which you can do if you'd like to try your own flat fielding. In this demo, we set skip_flat_field=False to use the standard flat fielding included in Eureka!.
#### 3. testing_S2 - Setting this keyword to True will run Stage 2 on a subset of the data so that it runs faster. You can use this if you're making your own edits to Stage 2 and want to quickly make sure you don't have any errors.
#### 4. hide_plots - If you set this to True, the plots will be saved into your directory but won't print onto the screen.
#### 5. topdir - Edit this to make it the path to where you've downloaded the data for this demo. The path should look like this: /path/to/NIRCam_demo/NIRCam_full_data/
#### 6. inputdir - This keyword tells Eureka! where to look for the Stage 1 outputs to feed into the Stage 2 code. This path is relative to "topdir". In this demo, we set inputdir=/Stage1/LongWavelengthSpectroscopy/
#### 7. outputdir - This is where Eureka! will save all the Stage 2 outputs, including plots and log files. Here we set outputdir=/Stage2/, which means the output files will be saved within a folder on the path /path/to/NIRCam_demo/NIRCam_full_data/Stage2/

In [ ]:
s2_meta = s2.calibrateJWST(eventlabel, ecf_path=ecf_path)

## Stage 3: Identify source position, perform background subtraction, perform spectral extraction

### From here on out, the remaining steps can be run on either the tiny or full data set (although note that the tiny data set does not contain a transit, so light curve fitting the tiny data set will probably produce weird results). If you are running the tiny data set, remove the keyword "s2_meta=s2_meta" from the line of code below. This keyword just tells Eureka! if you ran Eureka! on Stage 2 as well, which for the tiny data set you will not!

### This step contains a lot of the meat of going from a 2D image to a 1D extracted spectrum, so we'll go through most of the .ecf parameters individually.

#### 1. ncpu - Number of CPUs on your machine. This won't affect the data reduction except to make it run faster if you are able to run it on more CPUs.
#### 2. suffix - This specifies the suffix of the files you're feeding into Stage 3. For both the full and tiny data sets, the suffix for this demo is "calints".
#### 3. ywindow and xwindow - These specify the region of the image that you're interested in performing background subtraction and spectral extraction on. Note that this is the *full* image, not just the window surrounding the spectral trace. However, in many cases there may be reference pixels on the edges which you want to crop out - this is why the xwindow and ywindow in this demo select almost, but not quite, the entire image. I recommend opening a full image in ds9 or another fits file viewer to determine how to set these parameters. Reference pixels will show up as a dark rim along one/multiple edges of the image. You may also wish to cut the edges of the image beyond where you can see the spectral trace (i.e., where the throughput from the spectrograph drops to near nothing). However, you can also perform this cropping in Stage 4.
####  4. src_pos_type - Sets the method you'll use to locate the spectral trace. There are 3 options: gaussian (fit a gaussian), weighted (calculate the flux-weighted mean), or max (simply locate the pixel of maximum fluence in each column). You can try all 3, although for this example they shouldn't make a big difference.
### The next several parameters have to do with the background subtraction. Eureka! performs background subtraction by identifying the spectral trace, masking out a region surrounding the spectral trace, and using the remaining pixels as the background.
#### 5. bg_hw - Defines the half-width of the masked area not included in background subtraction. In this example, a value of 16 means that 16 pixels both above and below the identified source position are excluded from each column. I recommend testing a few different values ranging from quite close to the spectral trace to a bit farther away to determine which value gives the best errors in Stage 4. To test multiple values, enter an array in brackets containing the minimum aperture to test, maximum aperture to test, and step size. For example, [5,11,2] would test apertures of 5,7,9, and 11.
#### 6. bg_thresh - Defines the sigma value for a rejection of outliers along the time axis (i.e., rejecting outliers in the time series for a given pixel). There are two values because this sigma rejection is performed twice. For this example you shouldn't need to change this, but for other data sets it can be changed as needed to optimize cosmic ray removal.
#### 7. bg_deg - The background is subtracted by doing a column-by-column polynomial fit, and bg_deg defines the degree of that fit. Setting bg_deg = -1 will just calculate and subtract out a median for each column. For this example, bg_deg=1 proved good enough, but again this parameter can be changed to optimize background subtraction for other data sets.
#### 8. p3thresh - Defines the sigma value for a rejection of outliers *within* an image, as opposed to within a time series for a single pixel. Like with bg_thresh, this can be changed to optimize cosmic ray removal.
### Now some parameters for how we'll do the spectral extraction!
#### 9. spec_hw - Defines the half-width of the region you extract the spectrum from. In this example, a value of 8 means that the extraction will be perfomed on a box of pixels extending 8 up and 8 down from the identified source position. I recommend testing a few different values, just like for bg_hw, to see whether using a tight box around the maximum source flux or a wide box including all the less saturated pixels works better. As with bg_hw, you can use the Stage 4 errors to assess what width is best. *Note that spec_hw should be <= bg_hw, as you probably don't want your source extraction region to overlap with your background region!* To test multiple values, enter an array in brackets containing the minimum aperture to test, maximum aperture to test, and step size. For example, [5,11,2] would test apertures of 5,7,9, and 11.
#### 10. fittype - The method used to fit the spatial profile of the spectrum (i.e., its extent in the column direction). The options that are enabled for this demo are meddata (construct the spatial profile using the median of all the data frames), smooth (using a smoothing function), and poly (using a polynomial fit in the wavelength direction). Feel free to test these out and see how it changes the resulting spectrum!
#### 11. window_len - If using fittype=smooth, this defines the length of the smoothing window.
#### 12. prof_deg - If using fittype=poly, this defines the degree of polynomial to be used for the fit.
#### 13. p5thresh - This defines the sigma threshhold for rejecting outliers during the step where you use fittype to make the spatial profile.
#### 14. p7thresh - This defines the sigma threshhold for rejecting outliers during the spectral extraction. Both this and p5thresh can be changed if your spectrum has unexpected outliers or if it's accidentally rejecting what you think is good data, but for this demo, 5 should work for both of them.
### Finally, some parameters for printing diagnostics and saving output.
#### 15. isplots_S3 - How many plots do you want to create? This can be set to 1, 3, or 5, where a bigger number will print out more different types of diagnostic plots. Although it makes a lot of output, I recommend setting this to 5 - this gives you the most ways to understand what happened in your data reduction through different plots.
#### 16. testing_S3 - Just like in Stage 2, setting this to True will make it run faster on a short subset of the data if you're testing your own modifications to the pipeline. For this demo, leave it at False.
#### 17. hide_plots - Same as in Stage 2.
#### 18. save_output - Leave this set to True to make sure the Stage 3 outputs are saved and ready to use for the rest of the steps in this demo.
#### 19. topdir - Same as in Stage 2.
#### 20. inputdir - Since we now want to use Stage 2 outputs as the Stage 3 inputs, set this to /Stage2/
#### 21. outputdir - Similar to Stage 2, here I've set it to /Stage3/
#### 22. ancildir - This is the directory that contains ancillary files used in Stage 3, such as the gain. The files used here are included in the demo download, so leave this variable set to /ancil/ so that Eureka! knows where to find them.

In [ ]:
s3_meta = s3.reduceJWST(eventlabel, ecf_path=ecf_path, s2_meta=s2_meta)

## Stage 4: Convert 1D extracted spectra to time series light curves

### This stage converts the extracted spectra from Stage 3 into light curves. It also produces plots and diagnostics measuring the precision of your extracted spectra, which you can use to fine-tune your extraction procedure. If using Stage 4 to diagnose the level of error in your extracted spectra, I recommend making a single white light curve summing over all wavelengths (we'll go through how to do this below) and looking at the median absolute deviation (MAD, calculated and printed on the Stage 4 plots). Here are the .ecf parameters:

#### 1. nspecchan - the number of spectral channels you want to divide your light curve into. The number in this demo (35) is set to match the specifications for the [ERS 2022 Data Challenge](https://gist.github.com/zkbt/8356845f54e31f12213224bf959e54ac), but removing wavelength bins which fall beyond the region of high throughput and don't have much flux in them. To make a white light curve, set nspecchan=1.
#### 2. wave_min - the minimum wavelength where your spectral bins will start. Again, this is set to match the ERS Data Challenge bins. To make a white light curve, set this to the minimum wavelength of the full region of good throughput.
#### 3. wave_max - the maximum wavelength where your spectral bins will end. Again set to match the ERS Data Challenge bins. To make a white light curve, set this to the maximum wavelength of the full region of good throughput.
#### 4. allapers - If set to true, this will run Stage 4 for all of the different apertures you may have tested in Stage 3. Here we only run a single aperture, but if you want to test several apertures this is a quick way to run all of them through Stage 4 at once.
### The next several parameters (correctDrift, all of the ones starting with drift_, sub_mean, sub_continuum, and highpassWidth) are for performing a drift correction. This is not necessary for the simulated data, so for now leave correctDrift=False and these will be skipped. To try these out on real data, check out the up-to-date description of these parameters in the [Eureka Documentation](https://eurekadocs.readthedocs.io/en/latest/ecf.html).
### The parameters sigma_clip, sigma, box_width, maxiters, and fill_value are used if you decide to perform sigma clipping to remove outliers from the time series. These parameters are all well described in the .ecf file itself, and you can leave them set the way they are for now. However, if you run into issues with outlier removal being too strict or too loose, try changing the sigma value.
#### 5. verbose - If this is set to true, it will print a more detailed narrative of what's happening in Stage 4 onto the screen.
### The remaining keywords (isplots_S4, hide_plots, topdir, inputdir, and outputdir) are identical to the ones in Stage 3. The only change is that inputdir should now be /Stage3/ and outputdir should be /Stage4/.

In [ ]:
s4_meta = s4.lcJWST(eventlabel, ecf_path=ecf_path, s3_meta=s3_meta)

## Stage 5: Light Curve Fitting

### This stage fits the lightcurves created in Stage 4 to produce best fi97 nbt parameters for the transit, as well as for any systematics. This stage actually has two .ecf files: "S5_nircam_wfss_template.ecf" and "S5_fit_par_template.ecf". "S5_nircam_wfss_template.ecf" functions similarly to the .ecf files for all of the earlier stages. "S5_fit_par_template.ecf" contains information on the parameters you'll be fitting the lightcurve with. Let's start with "S5_nircam_wfss_template.ecf".

#### 1. allapers - Like in Stage 4, if set to True this will run Stage 5 for all of the different apertures you tested in Stages 3/4. In this demo we are only running a single aperture.
#### 2. rescale_err - If set to true, this will rescale uncertainties so that each lightcurve has a reduced chi squared equal to 1. This can be useful if you are getting reduced chi squared values much larger than 1 and think that your fit is underestimating the real errors.
#### 3. fit_par - This tells the code the location of the fit parameters Eureka Control File. For this demo, the file is called "S5_fit_par_template.ecf".
#### 4. run_verbose - If set to True, many more updates about the status of the fit will be printed into your terminal as you run the code.
#### 5. fit_method - Sets which type of fit you want to use. The fitting types enabled in this demo are lsq (least squares), emcee (MCMC fitting with the [emcee](https://emcee.readthedocs.io/en/stable/) package), and dynesty (nested sampling with the [dynesty]{https://dynesty.readthedocs.io/en/stable/} package).
#### 6. run_myfuncs - Lists all of the astrophysical and systematic functions you will fit to the data. The .ecf file lists all of the available options, but for this demo we will only use the following two functions: batman_tr (fits a transit with the [batman](https://lweb.cfa.harvard.edu/~lkreidberg/batman/) package) and polynomial (can generally fit a degree n polynomial - for this demo we'll just be fitting a constant offset, or a polynomial of degree 0)
### The next two parameters will eventually be implemented to enable you to use fixed limb darkening outputs from a limb darkening calculator, but they are not available for this demo.
### After this, the .ecf file lists sets of parameters unique to different fitting methods you may choose. These parameters will be ignored if you're using a different fitter, so you only need to worry about the parameters for whichever fit you're using. First, least squares parameters:
#### 7. lsq_method - The least squares fitting uses the scipy.optimize.minimize method, which can perform the fit with several types of solvers. 'Nelder-Mead' is the default, but if you want to try a different method you can find the full list in the [scipy.optimize.minimize documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html).
#### 8. lsq_tol - The tolerance for terminating the fit. We'll leave this fixed to the standard value of 1e-6.
### Next are parameters for if you're running an emcee fit.
#### 9. lsq_first - Setting this to True will initialize the mcmc fit with a least squares fit. If you're having problems with the initial least squares fit driving you into the wrong corner of parameter space, set this to False instead and just provide good inital parameter guesses to the mcmc.
#### 10. run_nsteps - Total number of steps to run the mcmc for. If your fit isn't converging, use more steps.
#### 11. run_nwalkers - The number of walkers to use in the mcmc fit. 100 should be sufficient for the number of parameters we have here.
#### 12. run_nburn - How many steps to discard as part of the burn-in. Usually around 10% of the total number of steps is good.
### Finally, we have some parameters for running a dynesty fit.
#### 13. run_nlive - Number of live points.
#### 14. run_bound - Type of bounding for the target distribution. The default for dynesty is 'multi', but check the dynesty documentation for all the available options.
#### 15. run_sample - Sampling method to use. Check the dynesty documentation for the available options.
#### 16. run_tol - Tolerance for ending the fit.
### The final parameters are general to all fit types.
#### 17. interp - If True, the final fit will be interpolated between the times of the data points for plotting purposes. This can be useful for data sets such as for Hubble, where there are large gaps in the observations.
### Almost all of the remaining parameters (isplots_S5, testing_S5, hide_plots, topdir, inputdir, and outputdir) are identical to those in Stage 3. The only difference is in this case the inputdir is /Stage4/ and the outputdir is /Stage5/. And there is one more parameter:
#### 18. testing_model - If True, this will run a fit for a single systematics model without also fitting a transit. This can be used to test adding in more systematics models if your simulated data don't actually have a transit included.

### Ok, that was a lot of parameters! But now let's take a look at the second file, "S5_fit_par_template.ecf". This file contains information on each of the parameters you're fitting for. Each parameter has the following information: Name, Starting Value, Free?, PriorPar1, PriorPar2, and PriorType. The name and starting value are straightforward. The allowed values in the "Free?" column are:
#### 1. 'free' - This indicates a free parameter that will be fit for.
#### 2. 'fixed' - This indicates a parameter that will be fixed to the given value and not fit for.
#### 3. 'independent' - This is used specifically for the parameters time_offset, limb_dark, and transittype, which feed into fitting the data but are not actually parameters that could be fit for by any of the fitting functions.
#### 4. 'shared' - If you are fitting multiple lightcurves simultaneously,  this indicates a parameter that will be fit for but will be constrained to have the same value across all light curves. This could be used if, for example, you had several light curves at different wavelengths and wanted to fit a mid-transit time for all of them but wanted all light curves to have the same transit time. NOTE: Currently, using this option for any parameters greatly increases the time for fits to run, especially for emcee and dynesty. This will be improved in a future version of Eureka!.
### The parameters PriorPar1, PriorPar2, and PriorType describe what type of prior will be used for each free parameter. There are 3 options:
#### 1. PriorType = U - This is a uniform prior. In this case, PriorPar1 is the lower limit and PriorPar2 is the upper limit.
#### 2. PriorType = LU - This is a log uniform prior. In this case, PriorPar1 is the lower limit and PriorPar2 is the upper limit.
#### 3. PriorType = N - This is a normal (Gaussian) prior. In this case, PriorPar1 is the mean and PriorPar2 is the standard deviation.
### Here is a description of the parameters themselves. The first several parameters are for the transit model:
#### 1. rp - planet-to-star radius ratio, unitless
#### 2. per - period, in days
#### 3. t0 - mid-transit time, in Julian Days - 2,400,000
#### 4. time_offset - used for plotting purposes. For this demo, leave this set to 0.
#### 5. inc - inclination, in degrees
#### 6. a - ratio of the semi-major axis to the stellar radius, unitless
#### 7. ecc - eccentricity, unitless
#### 8. w - longitude of periastron, in degrees
#### 9. limb_dark - What type of limb darkening model you are fitting for. The options are 'uniform', 'linear', 'quadratic', 'square-root', 'logarithmic', 'exponential', 'power2', and 'nonlinear'. See the batman documentation for a full description of each model.
#### 10. u1, u2, etc. - Parameters that start with "u" are the parameters that are fed into the limb darkening model. The number of parameters depends on the model - i.e., a linear limb darkening model takes only one parameter, while a quadratic model takes two parameters.
#### 11. transittype - 'primary' indicates observing a primary transit; 'secondary' indicates observing a secondary eclipse
### Finally, you can add parameters to the template based on which systematics models you're using.
#### 12. c0, c1, etc. - These parameters are for the polynomial model, and you can change the number based on the degree of polynomial you're fitting. For example, a degree 2 polynomial would have parameters c0, c1, and c2.
### The other models are not used in this demo, so their parameters aren't defined here. However, check out the Eureka! documentation if you want to use any other models.

In [ ]:
s5_meta = s5.fitJWST(eventlabel, ecf_path=ecf_path, s4_meta=s4_meta)

## Stage 6: Create Final Spectrum

### This stage takes the fitted light curves from Stage 5 and outputs a plot showing the final transmission spectrum. Let's go through the .ecf parameters.

#### 1. allapers - This is the same as in Stages 4/5.
#### 2. y_unit - The unit to plot on the y-axis. Can be Rp/Rs or (Rp/Rs)^2 for a transit, or Fp/Fs for an eclipse.
#### 3. y_scalar - Set equal to 100 to plot percent, 10^6 to plot ppm, etc.
#### 4. x_unit - The unit to plot on the x-axis. This can be any recognized measurement of wavelength/frequency, such as micron (um), nanometer (nm), Hertz (Hz), etc.
### The next several parameters are used if you want to plot the transit depths in terms of scale height.
#### 5. star_Rad - Radius of the star, units of solar radii.
#### 6. planet_Teq - Equilibrium temperature of the planet, units of Kelvin.
#### 7. planet_Mass - Mass of the planet, units of Jupiter masses.
#### 8. planet_Rad - Radius of the planet, in units of Jupiter radii. If set to None, it will calculate the average planet radius from the fitted light curves and use that.
#### 9. planet_mu - Mean molecular mass of the atmosphere, in atomic mass units. For a H/He-dominated atmosphere, 2.3 is appropriate.
#### 10. planet_R0 - Reference radius to use for the scale height calculation, units of Jupiter radii. If set to None, it will use the mean planet radius from the fitted light curves.
### Most of the remaining parameters (isplots_S6, testing_S6, hide_plots, topdir, inputdir, and outputdir) are identical to previous Stages. For this stage, inputdir is /Stage5/ and outputdir is /Stage6/. The remaining parameters other than these ones are all for plotting a model to compare to a spectrum. We won't do this in this demo, but you can check out the full Eureka! documentation for more information on how to implement this.

In [ ]:
s6_meta = s6.plot_spectra(eventlabel, ecf_path=ecf_path, s5_meta=s5_meta)